In [2]:
pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 219.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 590.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 395.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 419.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.9/434.9 kB 411.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.3/142.3 kB 488.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.3/126.3 kB 445.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 573.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 378.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 439.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 211.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.0/130.0 kB 518.6 MB/s eta 0:00:00
     ━━━━━

In [3]:
pip install rdkit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.1/33.1 MB 187.7 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
import gradio as gr
import openvino.runtime as ov
from rdkit import Chem
from rdkit.Chem import AllChem, Draw, rdMolDescriptors
import numpy as np
import torch
import intel_extension_for_pytorch as ipex  # Import IPEX
import random
from PIL import Image

# Define the PyTorch model (assuming `Net` is your model class)
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = torch.nn.Linear(2048, 1024)
        self.dropout1 = torch.nn.Dropout(0.3)
        self.fc2 = torch.nn.Linear(1024, 512)
        self.dropout2 = torch.nn.Dropout(0.3)
        self.fc3 = torch.nn.Linear(512, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout1(x)
        x = torch.relu(self.fc2(x))
        x = self.dropout2(x)
        return self.fc3(x)

# Load and optimize the PyTorch model with IPEX
torch_model = Net()
torch_model.load_state_dict(torch.load('lipophilicity_model.pth'))
torch_model.eval()
ipex_model = ipex.optimize(torch_model)  # IPEX-optimized PyTorch model

# Load the OpenVINO model
core = ov.Core()
ov_model_path = 'binding_affinity_model_openvino.xml'  # Path to your OpenVINO model
compiled_ov_model = core.compile_model(core.read_model(ov_model_path), "CPU")

# Function to convert SMILES to a Morgan fingerprint
def smiles_to_fp(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        raise ValueError("Invalid SMILES string.")
    fp = rdMolDescriptors.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=2048)
    return np.array(fp, dtype=np.float32)

# Function to score a molecule using either OpenVINO or IPEX
def score_molecule(mol, method="OpenVINO"):
    # Convert molecule to fingerprint
    fp_array = smiles_to_fp(Chem.MolToSmiles(mol))
    input_tensor = torch.tensor(fp_array, dtype=torch.float32).unsqueeze(0)

    if method == "OpenVINO":
        # Run OpenVINO inference
        ov_input_tensor = ov.Tensor(input_tensor.numpy())
        result = compiled_ov_model([ov_input_tensor])[0]
        return result[0].item()
    else:  # IPEX inference
        with torch.no_grad():
            result = ipex_model(input_tensor)
        return result.item()

# Step 1: Generate New Molecules by Modifying Core Scaffold
def generate_new_molecules(smiles, num_variants=5):
    scaffold_mol = Chem.MolFromSmiles(smiles)
    new_molecules = []
    functional_groups = ["C", "CC", "O", "N", "Cl"]

    for _ in range(num_variants):
        mol_copy = Chem.RWMol(scaffold_mol)
        atom_idx = random.randint(0, mol_copy.GetNumAtoms() - 1)
        func_group_smiles = random.choice(functional_groups)
        func_group_mol = Chem.MolFromSmiles(func_group_smiles)
        
        combined_mol = Chem.CombineMols(mol_copy, func_group_mol)
        combined_mol = Chem.RWMol(combined_mol)
        
        try:
            AllChem.SanitizeMol(combined_mol)
            new_smiles = Chem.MolToSmiles(combined_mol)
            new_molecules.append(Chem.MolFromSmiles(new_smiles))
        except:
            pass
    
    return new_molecules

# Step 2: Generate Conformers for Each New Molecule
def generate_conformers(mol, num_conformers=10):
    mol = Chem.AddHs(mol)
    AllChem.EmbedMultipleConfs(mol, numConfs=num_conformers, randomSeed=42)
    return mol

# Step 3: Score Conformers Using Selected Method
def score_conformers(mol, method="OpenVINO"):
    scores = []
    for conf_id in range(mol.GetNumConformers()):
        score = score_molecule(mol, method)
        scores.append((conf_id, score))
    return scores

# Drug Discovery Pipeline Function
def drug_discovery_pipeline(initial_smiles, num_molecules=5, num_conformers=10, method="OpenVINO"):
    new_molecules = generate_new_molecules(initial_smiles, num_molecules)
    best_molecules = []
    
    for mol in new_molecules:
        mol_with_confs = generate_conformers(mol, num_conformers)
        conformer_scores = score_conformers(mol_with_confs, method)
        conformer_scores = sorted(conformer_scores, key=lambda x: x[1])
        
        best_conformer_id = conformer_scores[0][0]
        best_score = conformer_scores[0][1]
        best_molecules.append((mol, best_conformer_id, best_score))
    
    best_molecules = sorted(best_molecules, key=lambda x: x[2])
    return best_molecules[:3]

# Visualization Function
def visualize_molecules(molecules):
    images = []
    for mol, conf_id, score in molecules:
        img = Draw.MolToImage(mol, kekulize=True)
        images.append((img, f"Score: {score:.4f}, Conformer ID: {conf_id}"))
    return images

# Gradio Function to Run Pipeline and Display Results
def run_pipeline(initial_smiles, num_molecules, num_conformers, method):
    top_molecules = drug_discovery_pipeline(initial_smiles, num_molecules, num_conformers, method)
    return visualize_molecules(top_molecules)

# Gradio Interface
interface = gr.Interface(
    fn=run_pipeline,
    inputs=[
        gr.Textbox(label="Initial SMILES", placeholder="e.g., C1=CC=CC=C1"),
        gr.Slider(label="Number of Molecules", minimum=1, maximum=10, step=1, value=5),
        gr.Slider(label="Number of Conformers", minimum=1, maximum=20, step=1, value=10),
        gr.Radio(["OpenVINO", "IPEX"], label="Inference Method", value="OpenVINO")
    ],
    outputs=gr.Gallery(label="Top Molecules and Scores"),
    title="Drug Discovery Pipeline",
    description="Generate and score new molecules based on an initial scaffold using OpenVINO or IPEX and RDKit."
)

interface.launch(share=True)


/tmp/ipykernel_1435/2116171809.py:30: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch_model.load_state_dict(torch.load('lipophilicity_model.pth'))


Running on local URL:  http://127.0.0.1:7861
Running on public URL: https://654ed52fa8b83f93da.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "/opt/app-root/lib64/python3.9/site-packages/torch/_inductor/compile_worker/__main__.py", line 45, in <module>
    main()
  File "/opt/app-root/lib64/python3.9/site-packages/torch/_inductor/compile_worker/__main__.py", line 38, in main
    pre_fork_setup()
  File "/opt/app-root/lib64/python3.9/site-packages/torch/_inductor/async_compile.py", line 62, in pre_fork_setup
    from triton.compiler.compiler import triton_key
ImportError: cannot import name 'triton_key' from 'triton.compiler.compiler' (/opt/app-root/lib64/python3.9/site-packages/triton/compiler/compiler.py)
W1107 12:21:51.838285 140495963436608 intel_extension_for_pytorch/utils/_logger.py:72] SubprocPool unclean exit


[12:22:26] DEPRECATION WARNING: please use MorganGenerator
[12:22:26] DEPRECATION WARNING: please use MorganGenerator
[12:22:26] DEPRECATION WARNING: please use MorganGenerator
[12:22:26] DEPRECATION WARNING: please use MorganGenerator
[12:22:26] DEPRECATION WARNING: please use MorganGenerator
[12:22:26] DEPRECATION WARNING: please use MorganGenerator
[12:22:26] DEPRECATION WARNING: please use MorganGenerator
[12:22:26] DEPRECATION WARNING: please use MorganGenerator
[12:22:26] DEPRECATION WARNING: please use MorganGenerator
[12:22:26] DEPRECATION WARNING: please use MorganGenerator
[12:22:26] DEPRECATION WARNING: please use MorganGenerator
[12:22:27] DEPRECATION WARNING: please use MorganGenerator
[12:22:27] DEPRECATION WARNING: please use MorganGenerator
[12:22:27] DEPRECATION WARNING: please use MorganGenerator
[12:22:27] DEPRECATION WARNING: please use MorganGenerator
[12:22:27] DEPRECATION WARNING: please use MorganGenerator
[12:22:27] DEPRECATION WARNING: please use MorganGenerat